In [ ]:
import mediapipe as mp
import cv2
import numpy as np
import math 

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)

cap = cv2.VideoCapture(0)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
while True:
    a = False 
    b = False 
    ret, frame = cap.read()
    
    frame = cv2.flip(frame, 1)
    
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    results = pose.process(frame_rgb)
    
    mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

    left_hip_x, left_hip_y = int(results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_HIP].x * width), int(results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_HIP].y * height)
    right_hip_x, right_hip_y = int(results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_HIP].x * width), int(results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_HIP].y * height)
    nose_x, nose_y = int(results.pose_landmarks.landmark[mp_pose.PoseLandmark.NOSE].x * width), int(results.pose_landmarks.landmark[mp_pose.PoseLandmark.NOSE].y * height)

        
    torso_length = abs(right_hip_y - left_hip_y)

    person_height = abs(nose_y - (left_hip_y + right_hip_y) / 2)

    center_to_hips_distance = abs(nose_y - (left_hip_y + right_hip_y) / 2)

    if center_to_hips_distance > person_height / 2:
        print("YES")
        b= True 
            
    cv2.imshow("Fall Detection", frame)
    
    if cv2.waitKey(1) == 27:
        break
    print('new')
    left_shoulder_x, left_shoulder_y = int(results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].x * width), int(results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].y * height)
    right_shoulder_x, right_shoulder_y = int(results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].x * width), int(results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].y * height)
    nose_x, nose_y = int(results.pose_landmarks.landmark[mp_pose.PoseLandmark.NOSE].x * width), int(results.pose_landmarks.landmark[mp_pose.PoseLandmark.NOSE].y * height)

    shoulder_angle = math.degrees(math.atan2(right_shoulder_y - left_shoulder_y, right_shoulder_x - left_shoulder_x))
    nose_angle = math.degrees(math.atan2(nose_y - left_shoulder_y, nose_x - left_shoulder_x))
    angle_diff = abs(shoulder_angle - nose_angle)

    if angle_diff < 10:
        print("Head and body are aligned horizontally.")
        a = True 
    else:
        print("Head and body are not aligned horizontally.")
    if (a == True & b == True ):
        print ("person fall ")
        
cap.release()
cv2.destroyAllWindows()
